In [1]:
import pandas as pd
import numpy as np
import sqlalchemy

from sqlalchemy_utils.functions import database_exists, create_database
from sqlalchemy.orm import declarative_base

In [2]:
PASSWORD = "root"
USERNAME = "root"
HOSTNAME = "localhost"
PORT = "3306"
DB_NAME = "data_lake_v2"
DATA_LAKE_FILEPATH = "./data/data_lake.csv"
SQLITE_ENGINE_DRIVER = f"sqlite+pysqlite:///{DB_NAME}"
MYSQL_ENGINE_DRIVER = f"mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DB_NAME}"


Creating SQLAlchemy Model

In [3]:
Base = declarative_base()


In [4]:
class PokemonCards(Base):
    __tablename__ = 'pokemon_cards'
    pokemon_card_id = sqlalchemy.Column("pokemon_card_id", sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column("name", sqlalchemy.String(length=70), nullable=False)
    card_number = sqlalchemy.Column("card_number", sqlalchemy.String(length=50), nullable=False)
    price = sqlalchemy.Column("price", sqlalchemy.Float, nullable=False)
    card_type = sqlalchemy.Column("card_type", sqlalchemy.String(200))
    generation = sqlalchemy.Column("generation", sqlalchemy.String(200))

    def __repr__(self):
        return f"(pokemon_card_id: {self.pokemon_card_id}, name:{self.name}, card_number:{self.card_number}, price: {self.price}, card_type: {self.card_type}, generation: {self.generation})"

Handle:
1. Engine Creation
2. Creating DB if not exist
3. Creating Tables within database

In [5]:
engine = sqlalchemy.create_engine(MYSQL_ENGINE_DRIVER, echo=False)

# Create DB if not exist
if not database_exists(engine.url):
    create_database(engine.url)


Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)


Importing data lake and start cleaning:
1. Retrieve all columns in relation to pokemons.
    1. Pokemon | Card Type | Generation | Card Number | Price 
2. Remove all nan values or replace them with valid data if necessary

In [6]:
columns_with_relation_to_pokemons = ['Pokemon', 'Card Number', 'Price', 'Card Type', 'Generation']
table_columns_name = [ 'name', 'card_number', 'price', 'card_type', 'generation']

data_lake = pd.read_csv(DATA_LAKE_FILEPATH)

pokemon_df = data_lake[columns_with_relation_to_pokemons]

C:\Users\joseh\AppData\Local\Temp\ipykernel_10920\704102097.py:4: DtypeWarning: Columns (3,4,6,7,8,9,10,11,15,16,17,18,21,22,24,25,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  data_lake = pd.read_csv(DATA_LAKE_FILEPATH)


In [7]:
# Cleaning NaN values | lower case strings | drop duplicates
pokemon_df = pokemon_df.dropna().apply(lambda x: x.str.lower() if x.dtype == "object" else x).drop_duplicates()

# Checking for any NaN value
pokemon_df.isna().sum()


Pokemon        0
Card Number    0
Price          0
Card Type      0
Generation     0
dtype: int64

Updating dataframe_column names to match values for our database table:

In [8]:
columns_mapping = dict(zip(columns_with_relation_to_pokemons, table_columns_name))
pokemon_df.rename(columns=columns_mapping, inplace=True)
pokemon_df.index = np.arange(1, len(pokemon_df) + 1)
pokemon_df.index.name = "pokemon_card_id"
pokemon_df

,name,card_number,price,card_type,generation
pokemon_card_id,,,,,
1,ampharos,001 of 147,2.95,reverse holo,aquapolis
2,ampharos,001 of 147,2.50,standard,aquapolis
3,arcanine,002 of 147,3.95,reverse holo,aquapolis
4,arcanine,002 of 147,2.95,standard,aquapolis
5,ariados,003 of 147,2.50,reverse holo,aquapolis
...,...,...,...,...,...
25426,mega gardevoir ex,112 of 114,7.85,standard,xy - steam siege
25427,pokemon ranger,113 of 114,4.65,standard,xy - steam siege
25428,professor sycamore,114 of 114,6.85,standard,xy - steam siege


Pushing data to DB

In [9]:
pokemon_df.to_sql(name="pokemon_cards", con=engine, if_exists="replace", index=True)

25430

Queries:
1. How many cards have been sold in total
2. How many "Ampharos" cards have been sold
3. How many different  generations labels are used in this database
4. Number of items that fall in a price range
5. What is the average price range for cards.

In [10]:
from sqlalchemy.orm import Session
from sqlalchemy import select


def execute_query(stmt) -> list:
    with Session(engine) as session:
        return [x[0] for x in session.execute(stmt)]

In [11]:
# How many cards have been sold in total
stmt_one = select(PokemonCards)
results = execute_query(stmt_one)
print(f"There have been {len(results)} cards sold")

There have been 25430 cards sold


In [12]:
# How many "Ampharos" cards have been sold
stmt_two = select(PokemonCards).where(PokemonCards.name == "ampharos")
results = execute_query(stmt_two)
print(f"There have been {len(results)} cards of Ampharos sold")


There have been 39 cards of Ampharos sold


In [13]:
# How many different  generations labels are used in this database
stmt_three = select(PokemonCards.generation).distinct()
results = execute_query(stmt_three)

print(f"There are {len(results)} labels for generations in this database")

There are 217 labels for generations in this database


In [14]:
# Number of items that fall in a price range
stmt_four = select(PokemonCards).filter(PokemonCards.price.between(1.90, 2.00))
results = execute_query(stmt_four)

print(f"There are {len(results)} that fall within that price range")


There are 382 that fall within that price range


In [15]:
# What is the average price range for cards.
stmt_five = select(sqlalchemy.func.round(sqlalchemy.func.avg(PokemonCards.price), 2))
results = execute_query(stmt_five)
print(f"The average sales price for pokemon cards is {results[0]}")

The average sales price for pokemon cards is 5.94
